In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "LLM Basics Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-82a9dc70' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=2e392738-eb54-4ee6-af45-4cae784cc467




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,3.277740,02c8fc33-9b2e-4699-9460-c2c7163fbb49,8efa2313-781f-48f1-ad92-f379ca127965
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you fir...",None,"To create a simple chain in LangChain, you can...",1,3.039599,21836737-355f-424c-a081-53d2416c5958,69e51f41-9472-455e-813c-a92d70c7ff3e
2,What is a vector store and why is it important...,A vector store is a type of database that hold...,None,A vector store is a database that stores embed...,1,2.864355,37ca6249-1656-4fc9-ad96-6b5335e63273,fc9e0f17-efb3-4d89-a4bd-9dbb7d7fad5b
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1,2.448310,607cba38-6f79-4617-91f5-69f809f60d7c,dba700a4-553f-4075-b390-6ca0af86b8a7
4,What is the difference between LangChain and L...,"LangChain is a set of open-source frameworks, ...",None,LangChain is an open-source framework for buil...,1,2.853047,6147b90b-c010-42f8-b028-c6ec1784ec9b,3f5895bc-296e-454b-b6d1-60dc494931a8
5,How do I track costs when using LLM APIs in pr...,To track costs when using LLM APIs in producti...,None,Track LLM API costs by monitoring token usage ...,1,2.853399,9bf9cdfc-7adb-4579-86e4-e6562c1ef569,b0a88077-d9df-4c91-9d47-0ecfda61cbee
6,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1,2.562297,d71e40e3-c281-4d0f-aca6-c4747292d34d,0f5f09e9-bcf7-4a7b-abbf-8c9055c28825
7,What are embeddings in the context of LLMs?,Embeddings in the context of LLMs (Large Langu...,None,Embeddings are dense vector representations of...,1,3.570524,d987ec00-a7b1-4614-98d5-222cc36e0861,94bec761-ab7a-4362-98f5-79bf3fee98bc
8,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1,3.256155,e99a8a6d-1be5-4571-abd4-f45ec65eff90,cd90b8e4-5266-42e3-9d75-ed012f0c5b54
9,What is the purpose of text splitting in RAG a...,The purpose of text splitting in RAG applicati...,None,Text splitting divides large documents into sm...,1,2.448835,f1644430-42bc-42e7-9ce4-561c764449ab,b1d1ce31-89c4-43ef-ae96-b4fb68a7707a


In [4]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-87b6224e' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=83cae50f-7263-417b-84c7-0e56340521fe




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is prompt engineering and why is it impor...,"Prompt engineering involves crafting, testing,...",None,Prompt engineering is the practice of crafting...,1,5.205854,02c8fc33-9b2e-4699-9460-c2c7163fbb49,1ce37b76-3f78-406d-ab17-96cae4cee6a9
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you nee...",None,"To create a simple chain in LangChain, you can...",1,4.700153,21836737-355f-424c-a081-53d2416c5958,37214461-494e-44f2-8ee1-e168fad67c79
2,What is a vector store and why is it important...,A vector store is a data structure used to sto...,None,A vector store is a database that stores embed...,1,3.062879,37ca6249-1656-4fc9-ad96-6b5335e63273,6839385c-00a8-4b8a-be34-61027b759139
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1,4.290337,607cba38-6f79-4617-91f5-69f809f60d7c,24571715-7d5c-4166-a8c5-a3c68ef60017
4,What is the difference between LangChain and L...,LangChain is an open-source framework focused ...,None,LangChain is an open-source framework for buil...,1,3.891317,6147b90b-c010-42f8-b028-c6ec1784ec9b,7e9f6578-c4e6-4739-9c7f-fe4c26c8dcf7
5,How do I track costs when using LLM APIs in pr...,To track costs when using LLM APIs in producti...,None,Track LLM API costs by monitoring token usage ...,1,3.064572,9bf9cdfc-7adb-4579-86e4-e6562c1ef569,31bff4c1-87b7-48f9-b7e2-e85e320878cc
6,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1,2.857966,d71e40e3-c281-4d0f-aca6-c4747292d34d,d26eb73c-4e93-4b9b-8a37-bcf2539a5449
7,What are embeddings in the context of LLMs?,Embeddings in the context of LLMs (Large Langu...,None,Embeddings are dense vector representations of...,1,3.268837,d987ec00-a7b1-4614-98d5-222cc36e0861,ab34b4e6-531f-4de0-974c-b33c1aff7c5c
8,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1,3.373259,e99a8a6d-1be5-4571-abd4-f45ec65eff90,5a2bc297-b4a1-4d45-bc9b-68170bfbf5de
9,What is the purpose of text splitting in RAG a...,The purpose of text splitting in RAG applicati...,None,Text splitting divides large documents into sm...,1,2.160258,f1644430-42bc-42e7-9ce4-561c764449ab,6eeca58a-0e6d-4f79-81ea-5887a2218109


In [11]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-8866e394' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=55332509-5d05-4cab-9edd-4d5676881720




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is the difference between LangChain and L...,LangChain is a set of open-source frameworks d...,None,LangChain is an open-source framework for buil...,1,3.276311,56be453a-1d51-48e2-93e1-83381c4b6ef3,09a78a16-10f1-4031-8b94-adf87777f9ad
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you sta...",None,"To create a simple chain in LangChain, you can...",1,3.204344,59b08d93-cfaf-47d4-b243-6674d7ffe71d,f5a57c07-a2af-472b-ba18-9586eddad3ae
2,How do I track costs when using LLM APIs in pr...,To track costs when using LLM APIs in producti...,None,Track LLM API costs by monitoring token usage ...,1,2.623872,5a57fafb-9113-4b95-9736-acb20fdcad13,7f602c3c-5f43-4d49-acfc-61c779e39d30
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1,2.334079,5e1f21b8-4325-4251-ad69-4301b2288b46,c63e95f2-2edb-417a-a06a-749fd8df1429
4,What is the purpose of text splitting in RAG a...,In Retrieval Augmented Generation (RAG) applic...,None,Text splitting divides large documents into sm...,1,2.573997,7d07364c-38d1-4a16-a913-eda4ddfa7a88,80493fe7-a1fc-4b2e-8ab3-0dba284b9ba0
5,What is a vector store and why is it important...,A vector store is a type of database that stor...,None,A vector store is a database that stores embed...,1,5.315848,a26d8699-0617-4aeb-a13a-543d6a264318,b106394d-70ee-4a62-93a5-72df42d516d9
6,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1,2.878018,afcf2fbc-99d1-46fe-8303-7ec528019d9c,d6437396-c684-4d4e-86cb-d21db0704aa9
7,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,2.845882,bfde0c48-ad01-4e26-95e6-9a10d68e537f,899b630b-2f40-42cd-9067-4e7eff0a2ced
8,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1,3.064520,cc7aa04d-a9f7-4bb0-a740-98505ec1948f,b1efb4e4-4629-4c02-889b-2e5c393f4ab5
9,What are embeddings in the context of LLMs?,Embeddings in the context of LLMs refer to den...,None,Embeddings are dense vector representations of...,1,2.766390,fc6149be-515b-48a5-be64-5e71a60d34d2,8a1d98b5-6d95-4554-85ff-0dbf3bbeeafc


In [6]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-2690ee51' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=6312f620-4d48-46f4-bcb2-f4448b105bc1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,6.935330,02c8fc33-9b2e-4699-9460-c2c7163fbb49,81c4dea1-7bc8-4db4-bbdd-c59d84c95a7f
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you fir...",None,"To create a simple chain in LangChain, you can...",1,3.883139,21836737-355f-424c-a081-53d2416c5958,5c854820-08f9-40ab-b003-ce32bcc4970b
2,What is a vector store and why is it important...,A vector store is a database that holds vector...,None,A vector store is a database that stores embed...,1,4.301593,37ca6249-1656-4fc9-ad96-6b5335e63273,9115daec-f286-423d-b1fa-6d756b22c4e2


In [7]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "37ca6249-1656-4fc9-ad96-6b5335e63273",
            "d987ec00-a7b1-4614-98d5-222cc36e0861"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-be9aed9e' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=85ff0ffe-409d-46f3-9df6-883301c332ec




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is a vector store and why is it important...,A vector store is a storage system that manage...,None,A vector store is a database that stores embed...,1,3.947568,37ca6249-1656-4fc9-ad96-6b5335e63273,37e8bf88-448d-4347-94da-f3288d52e973
1,What are embeddings in the context of LLMs?,I don't have the information on embeddings in ...,None,Embeddings are dense vector representations of...,1,2.655962,d987ec00-a7b1-4614-98d5-222cc36e0861,6ca679e6-7260-432e-9db5-06bb609ad283


In [8]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-4537c2c8' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=66a5cc40-5bef-411f-8e57-4688df05dc9f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,3.791136,02c8fc33-9b2e-4699-9460-c2c7163fbb49,dff09923-4d7f-4359-9eaa-208205a9c446
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you typ...",None,"To create a simple chain in LangChain, you can...",1,3.265928,21836737-355f-424c-a081-53d2416c5958,f306aae1-33be-4dc2-ae39-fc1d4b84dd91
2,What is a vector store and why is it important...,A vector store is a storage system for data th...,None,A vector store is a database that stores embed...,1,2.764394,37ca6249-1656-4fc9-ad96-6b5335e63273,f619f34f-1afd-4941-8c75-a4d8e438cda4
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1,3.781136,607cba38-6f79-4617-91f5-69f809f60d7c,3edbfd97-ca56-4918-a035-9e4549cde775
4,What is the difference between LangChain and L...,LangChain is an open-source framework used for...,None,LangChain is an open-source framework for buil...,1,3.271172,6147b90b-c010-42f8-b028-c6ec1784ec9b,010ce0c5-7e3b-45e9-ad41-220abbbd585f
...,...,...,...,...,...,...,...,...
75,What is a vector store and why is it important...,A vector store is a database that stores data ...,None,A vector store is a database that stores embed...,1,3.169316,a26d8699-0617-4aeb-a13a-543d6a264318,617093f8-dc20-472b-9d94-cdb95354e5bf
76,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1,2.460330,afcf2fbc-99d1-46fe-8303-7ec528019d9c,c1202c24-cf68-41be-a648-826008409d8c
77,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,2.452532,bfde0c48-ad01-4e26-95e6-9a10d68e537f,cc63049a-01bc-4634-856a-30cf6fb8f6af
78,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1,3.686056,cc7aa04d-a9f7-4bb0-a740-98505ec1948f,27341874-56ba-4971-993c-bb35f4b9bb9a


In [9]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-8094c6b4' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=82c4b9c5-52ab-428d-b513-922d7139b103




0it [00:00, ?it/s]

Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-MP8PNw1oF5i8PIpqgObq3Nr7 on tokens per min (TPM): Limit 30000, Used 29184, Requested 1723. Please try again in 1.814s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Traceback (most recent call last):
  File "c:\Users\Lokesh Pareek\Desktop\Testing2OpenAPI\venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "c:\Users\Lokesh Pareek\Desktop\Testing2OpenAPI\venv\Lib\site-packages\langsmith\run_helpers.py", line 693, in wrapper
    function_result = run_container["context"].run(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lokesh Pareek\AppData\Local\Temp\ipykernel_18816\862480817.py", line 5, in target_function
    return langsmith_rag(inputs["question"])
           ^^^^^^^^^^^^

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id,feedback.wrapper
0,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1.0,2.835820,02c8fc33-9b2e-4699-9460-c2c7163fbb49,5dc6339a-614d-424c-81fa-997412495e4b,NaN
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you typ...",None,"To create a simple chain in LangChain, you can...",1.0,2.998940,21836737-355f-424c-a081-53d2416c5958,1c39171a-8357-4551-9f33-7ae2c4ee3ac8,NaN
2,What is a vector store and why is it important...,A vector store is a storage system that holds ...,None,A vector store is a database that stores embed...,1.0,3.366736,37ca6249-1656-4fc9-ad96-6b5335e63273,24df9014-0463-4451-a62c-bfc411b33e69,NaN
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1.0,2.045278,607cba38-6f79-4617-91f5-69f809f60d7c,3e2aff28-5e8d-482e-a332-c29e9fcd6e4e,NaN
4,What is the difference between LangChain and L...,LangChain is an open-source framework designed...,None,LangChain is an open-source framework for buil...,1.0,2.900334,6147b90b-c010-42f8-b028-c6ec1784ec9b,e7956935-4896-48b5-9b57-9622b08df894,NaN
5,How do I track costs when using LLM APIs in pr...,To track costs when using LLM APIs in producti...,None,Track LLM API costs by monitoring token usage ...,1.0,2.550013,9bf9cdfc-7adb-4579-86e4-e6562c1ef569,dd57f22f-65ef-4b9e-a05d-8376b661b72d,NaN
6,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1.0,2.162202,d71e40e3-c281-4d0f-aca6-c4747292d34d,578cba5b-b2cc-4322-a656-afcf51f72ff1,NaN
7,What are embeddings in the context of LLMs?,Embeddings in the context of LLMs are numerica...,None,Embeddings are dense vector representations of...,1.0,2.346901,d987ec00-a7b1-4614-98d5-222cc36e0861,c680c608-7043-431f-9a8f-d8095688f86e,NaN
8,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1.0,2.974495,e99a8a6d-1be5-4571-abd4-f45ec65eff90,45020ecb-5899-47bd-b370-0c16fdf14798,NaN
9,What is the purpose of text splitting in RAG a...,"In RAG applications, text splitting is used to...",None,Text splitting divides large documents into sm...,1.0,2.498784,f1644430-42bc-42e7-9ce4-561c764449ab,70f8d2a3-d2ec-4e9b-9133-170e9c9d93f9,NaN


In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-361f94ac' at:
https://smith.langchain.com/o/23b06f1a-0bec-43ca-b5ce-981d9fa8164e/datasets/3b3a2968-d4c6-4433-bc5a-8519d6f7b1bb/compare?selectedSessions=ae64832f-4c1d-43c5-96d4-b571ad96217c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is prompt engineering and why is it impor...,"Prompt engineering is the process of crafting,...",None,Prompt engineering is the practice of crafting...,1,2.671956,02c8fc33-9b2e-4699-9460-c2c7163fbb49,8cb9184a-28e1-4299-a69d-92289f52ca3d
1,How do I create a simple chain in LangChain?,"To create a simple chain in LangChain, you fir...",None,"To create a simple chain in LangChain, you can...",1,2.675400,21836737-355f-424c-a081-53d2416c5958,7bef51e5-c062-4e2d-932c-cb79d62bebdd
2,What is a vector store and why is it important...,A vector store is a data structure used to ind...,None,A vector store is a database that stores embed...,1,3.234599,37ca6249-1656-4fc9-ad96-6b5335e63273,bdba5854-d6d3-4ce9-b323-041a23cacc9e
3,How do I handle errors and retries when callin...,To handle errors and retries when calling LLM ...,None,"When calling LLM APIs, implement error handlin...",1,2.666595,607cba38-6f79-4617-91f5-69f809f60d7c,af70ae52-a6cd-4a98-9565-393319c608aa
4,What is the difference between LangChain and L...,LangChain is an open-source framework for deve...,None,LangChain is an open-source framework for buil...,1,2.756123,6147b90b-c010-42f8-b028-c6ec1784ec9b,893dff20-e625-41b7-ae91-307098c82810
5,How do I track costs when using LLM APIs in pr...,To track costs when using LLM APIs in producti...,None,Track LLM API costs by monitoring token usage ...,1,2.482180,9bf9cdfc-7adb-4579-86e4-e6562c1ef569,86458bb6-5368-4a81-a03f-60b84ebe9cd9
6,How do I implement streaming responses from an...,To implement streaming responses from an LLM A...,None,"To implement streaming responses, use the stre...",1,2.537092,d71e40e3-c281-4d0f-aca6-c4747292d34d,63d567be-d17e-4809-91f4-5d4d4dff969a
7,What are embeddings in the context of LLMs?,Embeddings in the context of LLMs (Large Langu...,None,Embeddings are dense vector representations of...,1,3.281875,d987ec00-a7b1-4614-98d5-222cc36e0861,247630ab-78e1-4e65-a001-ed863aee7ccc
8,How do I make API calls with rate limiting in ...,To make API calls with rate limiting in Python...,None,To implement rate limiting for API calls in Py...,1,2.557572,e99a8a6d-1be5-4571-abd4-f45ec65eff90,2f6e44aa-0108-41d3-bc4b-96bdcc6bb61f
9,What is the purpose of text splitting in RAG a...,"In RAG applications, text splitting is used to...",None,Text splitting divides large documents into sm...,1,3.165003,f1644430-42bc-42e7-9ce4-561c764449ab,24c3eaa6-4912-4eb4-920c-07884e52664c
